# Necessary Steps

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!pip install -U ultralytics

In [4]:
import json
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil
from random import choice
import re
import csv
import zipfile
import pandas as pd
import glob
import math
import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry import Polygon
from ultralytics import YOLO
from PIL import Image
import requests
import time
import os
from ultralytics import YOLO
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.util import img_as_ubyte
from shapely.geometry import box

In [5]:
cd /home/core-stack/Documents/Ponds

/home/core-stack/Documents/Ponds


In [6]:
pwd = os.getcwd()
pwd

'/home/core-stack/Documents/Ponds'

# Download Data for Inference

In [ ]:
pwd

In [ ]:
gdf = gpd.read_file("Shapefiles/Masalia_mws.geojson")

In [ ]:
# Compute the bounding box
minx, miny, maxx, maxy = gdf.total_bounds

# Create a bounding box polygon
bounding_box = gpd.GeoDataFrame(geometry=[box(minx, miny, maxx, maxy)], crs=gdf.crs)

# Plot the geometry and the updated bounding box
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, color="blue", alpha=0.5, edgecolor="black")
bounding_box.plot(ax=ax, edgecolor="red", facecolor="none", linewidth=2)

plt.show()

In [ ]:
# Get the bounding box coordinates
minx, miny, maxx, maxy = gdf.total_bounds

# Define bounding box points
topLeft = [minx, maxy]
topRight = [maxx, maxy]
bottomRight = [maxx, miny]
bottomLeft = [minx, miny]

# Print the coordinates
print(f"topLeft = {topLeft}")
print(f"topRight = {topRight}")
print(f"bottomRight = {bottomRight}")
print(f"bottomLeft = {bottomLeft}")

NOT NEEDED ANYMORE - Give coordinated of the bounding box drawn on GEE

In [ ]:
"""
#coords for the bounding box (boipariguda)
topLeft = [82.07347488402685,18.57110542069261]
topRight = [82.60081863402685,18.57110542069261]
bottomRight = [82.60081863402685,18.9572939770273]
bottomLeft = [82.07347488402685,18.9572939770273]
"""

Zoom Level and folder name, where the output (image tiles) will be saved

In [ ]:
zoom_level = 18      # Zoom level
output_folder = "Data/Zoom18/Masalia"

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def getCoords(zoomLevel):
    topleft = deg2num(topLeft[1], topLeft[0], zoomLevel)  # (c,b)
    topright = deg2num(topRight[1], topRight[0], zoomLevel)  # (a,b)
    bottomright = deg2num(bottomRight[1], bottomRight[0], zoomLevel)  # (a,d)
    bottomleft = deg2num(bottomLeft[1], bottomLeft[0], zoomLevel)  # (c,d)
    xmin = min(topleft[0], topright[0], bottomleft[0], bottomright[0])
    xmax = max(topleft[0], topright[0], bottomleft[0], bottomright[0])
    ymin = min(topleft[1], topright[1], bottomleft[1], bottomright[1])
    ymax = max(topleft[1], topright[1], bottomleft[1], bottomright[1])
    return (xmin, xmax, ymin, ymax)

def download_map_tiles(base_url, output_folder, zoom_level, scale):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get start time
    start_time = time.time()

    # Iterate over each tile within the specified range
    xmin, xmax, ymin, ymax = getCoords(zoom_level)
    for x in range(xmin, xmax + 1):
        for y in range(ymin, ymax + 1):
            # Construct the URL for the current tile with scale=3 for 640x640
            tile_url = f"{base_url}&x={x}&y={y}&z={zoom_level}&scale={scale}"
            print(tile_url)
            try:
                # Send HTTP GET request to fetch the tile
                response = requests.get(tile_url)

                # Check if request was successful (status code 200)
                if response.status_code == 200:
                    # Save the tile image to a file in the output folder
                    filename = f"tile_{zoom_level}_{x}_{y}.png"
                    filepath = os.path.join(output_folder, filename)
                    with open(filepath, "wb") as f:
                        f.write(response.content)
                    print(f"Downloaded: {filename}")
                else:
                    print(f"Failed to download tile ({x}, {y}), HTTP status code: {response.status_code}")

            except Exception as e:
                print(f"Error downloading tile ({x}, {y}): {e}")

    # Get end time
    end_time = time.time()

    # Print the total execution time
    print(f"Total time taken: {end_time - start_time} seconds")

In [ ]:
base_url = "https://mt1.google.com/vt/lyrs=s"
scale = 1           # scale of 1 = 256*256 dimensional image

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created the folder: {output_folder}")
else:
    print(f"The folder already exists: {output_folder}")

In [ ]:
download_map_tiles(base_url, output_folder, zoom_level, scale)

# SAVE PREDICTIONS IN CSV

In [4]:
pwd

'/home/core-stack/Documents/Ponds'

Initialize model and parameters

In [5]:
model_path = "Models/Wells_best.pt"
model_path = os.path.join(pwd, model_path)

image_dir = "Data/Zoom18/Masalia"
#image_dir = os.path.join(pwd, image_dir)

csv_file = "CSV_Output/Masalia_Wells.csv"

zoom  = 18

In [6]:
# Define class-specific confidence thresholds
conf_thresholds = {
    'Wells': 0.71,
}

# Class names (ensure these match the order used in your model training)
class_names = [
    'Wells',

]

# Mapping of class names to abbreviations
class_abbreviations = {
    'Wells': 'W',

}

Load the model

In [7]:
my_new_model = YOLO(model_path)

FUNCTIONS

In [9]:
def process_image(image_path, conf_thresholds):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return None, None, None, None, None

    results = my_new_model.predict(img)

    polygons = []
    pred_classes = []
    conf_scores = []

    if results[0].masks is not None:
        for i, (polygon, cls, conf) in enumerate(zip(results[0].masks.xy, results[0].boxes.cls.cpu().numpy(), results[0].boxes.conf.cpu().numpy())):
            class_name = class_names[int(cls)]
            if conf >= conf_thresholds[class_name]:
                polygons.append(polygon)
                pred_classes.append(class_name)
                conf_scores.append(conf)

    # Print only when detections are made
    if len(polygons) == 0:
        print(f"No detections in {image_path}")
    else:
        print(f"Detections found in {image_path}: {len(polygons)} polygons")

    return image_path, len(polygons), polygons, pred_classes, conf_scores


def extract_xtile_ytile(image_path):
    try:
        basename = os.path.basename(image_path)
        parts = basename.split('_')
        if len(parts) >= 4:
            xtile = int(parts[2])
            ytile = int(parts[3].split('.')[0].split()[0])
            return xtile, ytile
        else:
            raise ValueError("Filename does not contain valid tile coordinates")
    except Exception as e:
        raise ValueError(f"Filename {image_path} does not contain valid tile coordinates: {e}")


def tile_corners_to_latlon(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad_nw = math.atan(math.sinh(math.pi * (1 - 2 * (ytile / n))))
    lat_deg_nw = math.degrees(lat_rad_nw)

    lat_rad_se = math.atan(math.sinh(math.pi * (1 - 2 * ((ytile + 1) / n))))
    lat_deg_se = math.degrees(lat_rad_se)

    lat_deg_nw = max(min(lat_deg_nw, 85.0511), -85.0511)
    lat_deg_se = max(min(lat_deg_se, 85.0511), -85.0511)

    top_left = (lat_deg_nw, lon_deg)
    top_right = (lat_deg_nw, lon_deg + (360.0 / n))
    bottom_right = (lat_deg_se, lon_deg + (360.0 / n))
    bottom_left = (lat_deg_se, lon_deg)

    return top_left, top_right, bottom_left, bottom_right

def calculate_tile_center(top_left, top_right, bottom_left, bottom_right):
    center_lat = (top_left[0] + bottom_left[0]) / 2
    center_lon = (top_left[1] + top_right[1]) / 2
    return (center_lat, center_lon)


Predictions

In [10]:
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

In [11]:
len(image_files)

75626

In [ ]:
# Start timing
start_time = time.time()

max_vertices = 0
for image_path in image_files:
    _, _, polygons, _, _ = process_image(image_path, conf_thresholds)
    for polygon in polygons:
        max_vertices = max(max_vertices, len(polygon))

with open(csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    header = ["Image Path", "Predicted Class", "Center Latitude", "Center Longitude",
              "Top Left Latitude", "Top Left Longitude", "Top Right Latitude", "Top Right Longitude",
              "Bottom Left Latitude", "Bottom Left Longitude", "Bottom Right Latitude", "Bottom Right Longitude"]
    for i in range(1, max_vertices + 1):
        header.extend([f'X_{i}', f'Y_{i}'])
    csvwriter.writerow(header)

    for image_path in image_files:
        try:
            image_path, num_polygons, polygons, pred_classes, _ = process_image(image_path, conf_thresholds)
            if image_path is None:
                continue

            xtile, ytile = extract_xtile_ytile(image_path)
            top_left, top_right, bottom_left, bottom_right = tile_corners_to_latlon(xtile, ytile, zoom)
            latitude, longitude = calculate_tile_center(top_left, top_right, bottom_left, bottom_right)

            for pred_class, polygon in zip(pred_classes, polygons):
                row = [image_path, pred_class, latitude, longitude, top_left[0], top_left[1], top_right[0], top_right[1],
                       bottom_left[0], bottom_left[1], bottom_right[0], bottom_right[1]]
                for point in polygon:
                    row.extend([point[0], point[1]])
                csvwriter.writerow(row)
        except ValueError as e:
            print(e)
            continue

# End timing
end_time = time.time()

# Print the time taken
time_taken = end_time - start_time
print(f"CSV file '{csv_file}' saved successfully.")
print(f"Time taken to complete: {time_taken:.2f} seconds.")


0: 256x256 (no detections), 12.2ms
Speed: 2.4ms preprocess, 12.2ms inference, 32.6ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194552_112843.png

0: 256x256 (no detections), 12.4ms
Speed: 0.6ms preprocess, 12.4ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194501_112943.png

0: 256x256 (no detections), 12.3ms
Speed: 0.4ms preprocess, 12.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194664_112877.png

0: 256x256 (no detections), 13.1ms
Speed: 0.5ms preprocess, 13.1ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194592_113016.png

0: 256x256 (no detections), 18.9ms
Speed: 0.5ms preprocess, 18.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194583_112869.png

0: 256x256 (no det

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 256x256 (no detections), 11.7ms
Speed: 0.4ms preprocess, 11.7ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194415_112940.png

0: 256x256 (no detections), 12.4ms
Speed: 0.4ms preprocess, 12.4ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194452_112912.png

0: 256x256 (no detections), 12.9ms
Speed: 0.4ms preprocess, 12.9ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194392_112956.png

0: 256x256 (no detections), 34.5ms
Speed: 0.4ms preprocess, 34.5ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194556_112952.png

0: 256x256 (no detections), 26.6ms
Speed: 0.6ms preprocess, 26.6ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194619_112947.png

0: 256x256 3 Wellss,

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



No detections in Data/Zoom18/Masalia/tile_18_194514_112901.png

0: 256x256 (no detections), 8.2ms
Speed: 0.4ms preprocess, 8.2ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194458_112918.png

0: 256x256 1 Wells, 13.8ms
Speed: 0.5ms preprocess, 13.8ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194677_113043.png

0: 256x256 2 Wellss, 18.1ms
Speed: 0.4ms preprocess, 18.1ms inference, 1.8ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194491_112943.png

0: 256x256 (no detections), 14.1ms
Speed: 0.4ms preprocess, 14.1ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194506_112952.png

0: 256x256 (no detections), 12.6ms
Speed: 0.4ms preprocess, 12.6ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/

...

0: 256x256 (no detections), 21.7ms
Speed: 0.4ms preprocess, 21.7ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194594_112899.png

0: 256x256 (no detections), 18.2ms
Speed: 0.8ms preprocess, 18.2ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194456_112948.png

0: 256x256 (no detections), 15.0ms
Speed: 0.5ms preprocess, 15.0ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194596_112956.png

0: 256x256 (no detections), 14.1ms
Speed: 0.7ms preprocess, 14.1ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194655_112964.png

0: 256x256 1 Wells, 14.0ms
Speed: 0.4ms preprocess, 14.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194473_112941.png

0: 256x256 1 Wells, 14.8ms
S

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 256x256 (no detections), 20.0ms
Speed: 0.5ms preprocess, 20.0ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194583_112836.png

0: 256x256 (no detections), 18.0ms
Speed: 0.5ms preprocess, 18.0ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194616_112969.png

0: 256x256 1 Wells, 16.7ms
Speed: 1.4ms preprocess, 16.7ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194540_112852.png

0: 256x256 (no detections), 13.5ms
Speed: 0.4ms preprocess, 13.5ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194612_112977.png

0: 256x256 (no detections), 12.0ms
Speed: 0.4ms preprocess, 12.0ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194373_112956.png

0: 256x256 2 Wellss, 12.6ms


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



No detections in Data/Zoom18/Masalia/tile_18_194504_112870.png

0: 256x256 (no detections), 17.2ms
Speed: 0.4ms preprocess, 17.2ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194637_112961.png

0: 256x256 1 Wells, 31.1ms
Speed: 1.2ms preprocess, 31.1ms inference, 3.8ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194457_112880.png

0: 256x256 1 Wells, 15.1ms
Speed: 0.3ms preprocess, 15.1ms inference, 1.5ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18/Masalia/tile_18_194512_112846.png

0: 256x256 2 Wellss, 13.3ms
Speed: 0.6ms preprocess, 13.3ms inference, 1.8ms postprocess per image at shape (1, 3, 256, 256)
Detections found in Data/Zoom18/Masalia/tile_18_194413_112964.png: 1 polygons

0: 256x256 (no detections), 12.8ms
Speed: 0.4ms preprocess, 12.8ms inference, 2.1ms postprocess per image at shape (1, 3, 256, 256)
No detections in Data/Zoom18

# Convert to Geometry


In [7]:
pwd

'/home/core-stack/Documents/Ponds'

In [8]:
# Constants
ZOOM_LEVEL = 18
EARTH_CIRCUMFERENCE_DEGREES = 360  # degrees


In [9]:
# Load the CSV file
df = pd.read_csv("CSV_Output/Masalia_Wells.csv")

df.rename(columns={'Predicted Class': 'Class'}, inplace=True)


In [10]:
def degrees_per_pixel(zoom):
    total_pixels = 256 * (2 ** zoom)
    return EARTH_CIRCUMFERENCE_DEGREES / total_pixels

def pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, img_width, img_height):
    lon_range = lon_bottom_right - lon_top_left
    lat_range = lat_top_left - lat_bottom_right  # Note: latitude decreases as you go south
    lon = lon_top_left + (x / img_width) * lon_range
    lat = lat_top_left - (y / img_height) * lat_range  # y increases downward in image coordinates
    return lon, lat

# Find the maximum index for X_n and Y_n columns
x_columns = [col for col in df.columns if col.startswith('X_')]
if x_columns:
    max_index = max(int(col.split('_')[1]) for col in x_columns)
else:
    max_index = 0  # If no X_n columns are found

# Initialize an empty list to store GeoJSON features
geojson_features = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    lat_top_left = row['Top Left Latitude']
    lon_top_left = row['Top Left Longitude']
    lat_bottom_right = row['Bottom Right Latitude']
    lon_bottom_right = row['Bottom Right Longitude']

    tile_width, tile_height = 256, 256
    object_coords = []

    for i in range(1, max_index + 1):  # Dynamically set the range
        x_col = f'X_{i}'
        y_col = f'Y_{i}'
        if x_col in row and y_col in row:
            x = row[x_col]
            y = row[y_col]
            if pd.notna(x) and pd.notna(y):
                lon, lat = pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, tile_width, tile_height)
                if np.isfinite(lon) and np.isfinite(lat):
                    object_coords.append((lon, lat))

        if pd.isna(x) or pd.isna(y) or i == max_index:
            if object_coords:  
                centroid_lon = np.mean([coord[0] for coord in object_coords])
                centroid_lat = np.mean([coord[1] for coord in object_coords])
                centroid = Point(centroid_lon, centroid_lat)

                feature = {
                    "type": "Feature",
                    "geometry": centroid,
                    "properties": {"Class": row['Class']}
                }
                geojson_features.append(feature)
            object_coords = []

# Create a GeoDataFrame
geometries = [feature['geometry'] for feature in geojson_features]
properties = [feature['properties'] for feature in geojson_features]
gdf_final = gpd.GeoDataFrame(properties, geometry=geometries, crs="EPSG:4326")

# Transform and save
gdf_final = gdf_final.to_crs(epsg=3857)
gdf_final.to_file('Shapefile_OutputMasalia_Wells.shp')

# Print verification
print(gdf_final.head())
print(degrees_per_pixel(17))

   Class                         geometry
0  Wells  POINT (9682854.935 2784435.227)
1  Wells  POINT (9693560.896 2770428.142)
2  Wells  POINT (9681388.797 2755531.779)
3  Wells  POINT (9689764.842 2753536.693)
4  Wells   POINT (9696670.629 2756560.11)
1.0728836059570312e-05
